In [1]:
# xMD testing
import pandas as pd
import os
from xMD.xMD import xMD
from xMD.MD_Settings import GROMACS_Settings

settings = GROMACS_Settings()


/home/alexi/Documents/xMD


In [2]:




amber14sb_ff_path = os.path.join(os.getcwd())

# Set the GMXLIB environment variable
os.environ["GMXLIB"] = amber14sb_ff_path



In [3]:
settings = GROMACS_Settings()
settings.suffix = "APO_amber99"
# settings.search = "APO"

print(settings.config)

settings.topology = os.path.join(settings.topology,"BPTI")
print(settings.topology)
# make sure to turn on MPI for HPC 
settings.gmx_mpi_on = False


config
topology/BPTI


In [4]:
# list config dir 

configs = os.listdir(settings.config)

configs = [config if ("_" in config and ".mdp" in config) else None for config in configs]
configs = [config for config in configs if config is not None]

print(sorted(configs))

['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp', '7_prod_10ps_out.mdp', 'md_short.mdp']


In [5]:
# specify ARGS: -P, -R, -N
md = xMD(settings, 'BPTI_genvel1', "5PTI", 1)

mdps = ['1_minim.mdp', '2_equil.mdp', '3_equil.mdp', '4_equil.mdp', '5_equil.mdp', '6_equil.mdp', '7_prod.mdp']

file_names = ["5PTI_APO_amber99_H_solv_ions.gro", "5PTI_APO_amber99_H.top", "5PTI_APO_amber99_H.itp"]
# md.check_args()
md.create_directory_structure(overwrite=True)
md.run_experiment(config_files=mdps,topology_files=file_names)
save_path = md.save_experiment()

Replicate number:  1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Environment variables set:  GMXLIB /home/alexi/Documents/xMD
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Trial directory temporary:  temporary/MD/5PTI/BPTI_genvel1
Trial directory logs:  logs/MD/5PTI/BPTI_genvel1
Trial directory data:  data/MD/5PTI/BPTI_genvel1
Trial directory visualisation:  visualisation/MD/5PTI/BPTI_genvel1
Trial directory analysis:  analysis/MD/5PTI/BPTI_genvel1
Creating directory:  R_1 Creating directory:  R_2 Creating directory:  R_3 

checking input for internal consistency...

Setting the LD random seed to -537937418

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'NA'

Excluding 1 bonded neighbours molecule type 'CL'

processing coordinates...

double-checking input for internal consistency...

Reading position restraint coords from data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro

converting bonded parameters...

initialising group options...
Analysing residue names:
There are:    58    Protein residues
There are:  6503      Water residues
There are:    44        Ion residues
Analysing Protein...

The largest distance between excluded atoms is 0.417 nm between atom 208 and 209
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 52x52x

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx grompp -f config/1_minim.mdp -c data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro -p data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H.top -o data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_0.tpr -r data/MD/5PTI/BPTI_genvel1/R_1/5PTI_APO_amber99_H_solv_ions.gro -maxwarn 1 -v

Ignoring obsolete mdp entry 'ns-type'

NOTE 1 [file config/1_minim.mdp]:
  You have set rlist larger than the interaction cut-off, but you also have
  verlet-buffer-tolerance > 0. Will set rlist using
  verlet-buffer-tolerance.


NOTE 2 [file config/1_minim.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 3 [file config/1_minim.mdp]:
  Setting pcoupl from 'Parrinello-Rahman' to 'no'. Pressure coupling does
  not apply to steep.

Generating 1-4 interactions: f

Trajectory number:  1
checking input for internal consistency...

Setting the LD random seed to -319439505

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Reading position restraint coords from data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_0.gro

The center of mass of the position restraint coord's is  2.955  2.975  2.938

The center of mass of the position restraint coord's is  2.955  2.975  2.938

converting bonded parameters...

initialising group options...

renumbering atomtypes...
processing index file...
Making dummy/rest group for Acc. not used containing 20445 elements
Making dummy/rest group for Freeze containing 20445 elements
Making dummy/rest group for Energy Mon. containing 20445 elements
Number of degrees of freedom in T-Coupling group System is 41385.00
Making dummy/rest group for User1 containing 20445 elements
Making dummy/rest group for User2 containing 20445 elements
Making dummy/rest group for Compressed X containing 20445 elements
Making dummy/rest group for Or. Res. Fit containing 20445 elements
Making dummy/rest group for QMMM containing 20445 elements
T-Coupling       has 1 element(s): System
Energy Mon.      has 1 element(s): rest
Acc. not used    has 1 element(s): rest
Freeze           has 1 element(s): rest
User1            has 1 element(s): rest
User2            has 1 element(s): rest
VCM              has 1 element(s): System
Compressed X     has 1 element(s): rest
Or. Res. Fit     has 1 element(s): rest
QMMM      

...


Back Off! I just backed up data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_5.log to data/MD/5PTI/BPTI_genvel1/R_1/#APO_amber99_5PTI_5.log.2#
Reading file data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_5.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 50, rlist from 0.8 to 0.93

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 20 OpenMP threads 


Back Off! I just backed up data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_5.xtc to data/MD/5PTI/BPTI_genvel1/R_1/#APO_amber99_5PTI_5.xtc.2#

Back Off! I just backed up data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_5.edr to data/MD/5PTI/BPTI_genvel1/R_1/#APO_amber99_5PTI_5.edr.2#
starting mdrun 'TRYPSIN INHIBITOR'
100000 steps,    200.0 ps.
step 16850: timed with pme grid 52 52

Trajectory number:  6
checking input for internal consistency...

Setting the LD random seed to -608192545

processing topology...

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

turning H bonds into constraints...

Excluding 2 bonded neighbours molecule type 'SOL'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'NA'

turning H bonds into constraints...

Excluding 1 bonded neighbours molecule type 'CL'

turning H bonds into constraints...

processing coordinates...

double-checking input for internal consistency...

Setting gen_seed to -583794693

Velocities were taken from a Maxwell distribution at 300 K

converting bonded parameters...

initialising group options...
Analysing residue names:
There are:    58    Protein residues
There are:  6503      Water residues
There are:    44        Ion residues
Analysing Protein...

Changing nstlist from 10 to 50, rlist from 0.8 to 0.93

1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 1 rank on this node:
  PP:0,PME:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
PME tasks will do all aspects on the GPU
Using 1 MPI thread
Using 20 OpenMP threads 


Back Off! I just backed up data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.xtc to data/MD/5PTI/BPTI_genvel1/R_1/#APO_amber99_5PTI_6.xtc.2#

Back Off! I just backed up data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.edr to data/MD/5PTI/BPTI_genvel1/R_1/#APO_amber99_5PTI_6.edr.2#
starting mdrun 'TRYPSIN INHIBITOR'
5000000 steps,  10000.0 ps.
step 18050: timed with pme grid 52 52 52, coulomb cutoff 0.800: 55.5 M-cycles
step 18150: timed with pme grid 44 44 44, coulomb cutoff 0.897: 43.6 M-cycles
step 18250: timed with pme grid 40 40 40, coulomb cutoff 0.986: 48.1 M-cycles
step 18350: timed with pme grid 36 36 36, co

Running trjconv command 1:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.xtc', '-s', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr', '-pbc', 'mol', '-center', '-o', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-mol.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-nojump.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'
Running trjconv command 3:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-nojump.xtc', '-s', 'data/MD/5PTI/BPTI_genvel1/

TypeError: MD_Experiment.prepare_analysis() missing 1 required positional argument: 'tpr_path'

In [ ]:
md.prepare_analysis(tpr_path="data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr")


Running trjconv command 1:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.xtc', '-s', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr', '-pbc', 'mol', '-center', '-o', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-mol.xtc']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Running trjconv command 2:  ['gmx', 'trjconv', '-f', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-mol.xtc', '-s', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr', '-pbc', 'nojump', '-o', 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-nojump.xtc', '-dump', '0']
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 1: 'Protein'


                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/xMD
Command line:
  gmx trjconv -f data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.xtc -s data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr -pbc mol -center -o data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-mol.xtc

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr, VERSION 2023.2 (single precision)
Reading file data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 20445 elements
Group     1 (        Protein) has   892 elements
Group     2 (      Protein-H) has   454 elements
Group     3 (        C-alpha) has    58 elements
Group     4 (       Backbone) has   174 elements
Group     5 (      MainChain) has   233 elements
Group     6 (   MainChain+Cb) has   285 element

('data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-nojump.xtc',
 'data/MD/5PTI/BPTI_genvel1/R_1/APO_amber99_5PTI_6-nojump.pdb')

In [ ]:
def parse_gromacs_energy_log(filepath):
    """
    Parse a GROMACS log file to extract energy information at each step into a DataFrame.
    """
   # read the file
    with open(filepath, 'r') as f:
        log = f.readlines()
    
    # find the start of the energy table

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            # find:            Step           Time
            if len(split)==2 and split[0] == "Step" and split[1] == "Time":
                start = i
                break
            #find end

        for i, line in enumerate(log):
            #split by any whitespace
            split = line.split()
            if len(split)>1 and split[0] == "Statistics":
                end = i
                break

        # extract the table
        table = log[start+1:end]
    # extract the headers
    df = pd.DataFrame()

    return df


log_file_path = "data/MD/1K55/APO_MD60_genvel/R_1/APO_md_1K55_0.log"
# Parse the log file again with the updated function
energy_df_updated = parse_gromacs_energy_log(log_file_path)
